## 학습 데이터 준비
 txt파일의 카톡 데이터를 모델 학습에 편리한 형식으로 바꿔주는 작업을 합니다.

In [ ]:
import re
import pandas as pd
from google.colab import files
import io

uploaded = files.upload()

def katalk_msg_parse(file_path):
    my_katalk_data = list()
    katalk_msg_pattern = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2},.*:"
    date_info = "[0-9]{4}년 [0-9]{1,2}월 [0-9]{1,2}일 \S요일"
    in_out_info = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2}:.*"

    # uploaded is a dictionary, get the first file's content
    file_content = list(uploaded.values())[0]
    # Decode the content if it's in bytes and open it as a file-like object
    decoded_content = io.StringIO(file_content.decode('utf-8'))

    for line in decoded_content:  # Iterate over lines in the decoded content
        if re.match(date_info, line) or re.match(in_out_info, line):
            continue
        elif line == '\n':
            continue
        elif re.match(katalk_msg_pattern, line):
            line = line.split(",")
            date_time = line[0]
            user_text = line[1].split(" : ", maxsplit=1)
            user_name = user_text[0].strip()
            text = user_text[1].strip()
            my_katalk_data.append({
                                   'User': user_name,
                                   'Message': text
                                   })

        else:
            if len(my_katalk_data) > 0:
                my_katalk_data[-1]['Message'] += "\n"+line.strip()

    my_katalk_df = pd.DataFrame(my_katalk_data)

    return my_katalk_df

df=katalk_msg_parse(uploaded)

변경된 카톡 데이터

In [ ]:
pd.DataFrame(df)

## 데이터 형식 재조정

'나'와 '상대방'의 대화를 학습시켜야 하기 때문에
'me'와 'you'라는 형식의 데이터로 재조정합니다.

In [ ]:
new_df = pd.DataFrame(columns=['User', 'Message'])
current_user = None
current_message = ""

# 같은 User 가 연속으로 나타날 경우 한개의 Row로 처리
for index, row in df.iterrows():
    if current_user is None:
        current_user = row['User']
        current_message += row['Message']
    elif current_user != row['User']:
        # new_df = new_df.append({'User': current_user, 'Message': current_message}, ignore_index=True) 대신 아래 코드 사용
        new_df = pd.concat([new_df, pd.DataFrame([{'User': current_user, 'Message': current_message}])], ignore_index=True)
        current_user = row['User']
        current_message = row['Message']
    else:
        current_message += "\n" + row['Message']

# 마지막 행 추가
if current_user is not None:
    # new_df = new_df.append({'User': current_user, 'Message': current_message}, ignore_index=True) 대신 아래 코드 사용
    new_df = pd.concat([new_df, pd.DataFrame([{'User': current_user, 'Message': current_message}])], ignore_index=True)

#
me = new_df[new_df.User =='김도겸'].reset_index().Message
you = new_df[new_df.User =='김회훈'].reset_index().Message

data = pd.DataFrame({'me':me , 'you' : you})

# 모델 파인튜닝(미세조정)

모델을 내 대화에 맞게 조정하는 작업을 해봅시다.

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast

checkpoint = 'skt/kogpt2-base-v2'
device = 'cuda'

tokenizer = PreTrainedTokenizerFast.from_pretrained(checkpoint,
  bos_token='<s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

## 학습을 위한 데이터셋 만들기

학습 데이터셋을 만들기 위한 라이브러리를 설치합니다.

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(data).map(
    lambda x: {'text': f"### Me: {x['me']}\n### You: {x['you']}</s>" } #end token </s>
).train_test_split(test_size=0.1)

#kogpt2 의 경우 max length 가 1024로 1024 이상의 길이는 Truncate 해야줘야 합니다.
dataset = dataset.map(lambda x: tokenizer(x["text"] , truncation=True, max_length=1024, padding="max_length"), batched=True)

## 모델 불러오기
KoGPT2 모델을 사용해서 실습을 진행합니다.
https://github.com/SKT-AI/KoGPT2

In [ ]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(checkpoint).to(device)

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Tokenizer 설정
if tokenizer.pad_token is None:
    if tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token
    else:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    learning_rate=2e-5,
    optim="adamw_torch",
    fp16=True,  # GPU 지원 시
    logging_strategy="steps",
    logging_steps=100,
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# 학습 시작
trainer.train()

재조정된 모델에 입력을 도와주는 함수

In [ ]:
def chat_with_model(prompt: str, model, tokenizer, device='cuda', max_length=32) -> str:
    input_text = f"### Me: {prompt}\n### You: "

    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        return_token_type_ids=False
    ).to(device)

    output = model.generate(
        **inputs,
        max_length=max_length,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        use_cache=True,
        early_stopping=True,
        do_sample=True,
        temperature=0.7,
        top_k=30,
        top_p=0.80
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # '### You: ' 이후 텍스트 추출
    if '### You:' in decoded:
        return decoded.split('### You:')[1].strip()
    else:
        return decoded.strip()

# 이제는 직접 출력을 해봅시다!

In [ ]:
response = chat_with_model("카페갈건데 뭐 마실래??", model, tokenizer)
print(response)